## CIFAR 10

In [26]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [27]:
from fastai.conv_learner import *
from fastai.models.cifar10.preact_resnet import *
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [28]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512
sz=32
workers=7

In [5]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def pad(img, p=4, padding_mode='reflect'):
        return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))
def to_pil(img): return Image.fromarray(img)

def torch_tfms(size, conv_pil=False, to_numpy=False):
    mean,std=[0.4914 , 0.48216, 0.44653], [0.24703, 0.24349, 0.26159]
    normalize = transforms.Normalize(mean=mean, std=std)
    tfms = [transforms.ToTensor(), normalize]
    
    # Torch transforms with fastai dl
    if to_numpy: tfms = [np.array, Normalize(mean,std), lambda x: x[0].T]
        
    aug_tfms = [
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
    ]
    scale_size = 40
    padding = int((scale_size - size) / 2)
    
    train_tfms = transforms.Compose(aug_tfms + tfms)
    train_tfms.sz = size
    val_tfms = transforms.Compose(tfms)
    val_tfms.sz = size
    if conv_pil:
        train_tfms.transforms.insert(0, to_pil)
        val_tfms.transforms.insert(0, to_pil)
    return train_tfms, val_tfms
    
def torch_ds(data_path, tfms):
    train_tfms, val_tfms = tfms
    
    # Data loading code
    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'test')

    train_dataset = datasets.ImageFolder(traindir, train_tfms)
    val_dataset = datasets.ImageFolder(valdir, val_tfms)
    return train_dataset, val_dataset

def torch_loader(data_path, datasets):
    train_dataset, val_dataset = datasets
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs*2, shuffle=False,
        num_workers=workers, pin_memory=True)

    data = ModelData(data_path, train_loader, val_loader)
    return data


In [13]:
def get_learner(data):
    m = PreActResNet18()
    learn = ConvLearner.from_model_data(m, data)
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    return learn

### Torch Data Loader

In [14]:
tfms = torch_tfms(sz)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [15]:
learn = get_learner(torch_data)

In [16]:
# Warmup session
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      nan        nan        0.1       

CPU times: user 8.38 s, sys: 4.54 s, total: 12.9 s
Wall time: 15.9 s


[array([nan]), 0.1]

In [17]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 17.1 s, sys: 8.5 s, total: 25.6 s
Wall time: 26.9 s


[array([nan]), 0.1]

### Fastai Data Loader

In [18]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [19]:
learn = get_learner(fastai_data)

In [20]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                     
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 1min 50s, sys: 2min 59s, total: 4min 50s
Wall time: 47.7 s


[array([nan]), 0.1]

### Fastai Default Collate

In [29]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_dc_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [30]:
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = torch.utils.data.DataLoader(None).collate_fn

In [31]:
fastai_dc_data.trn_dl.collate_fn = collate_fn
fastai_dc_data.val_dl.collate_fn = collate_fn

In [32]:
learn = get_learner(fastai_dc_data)

In [33]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

  0%|          | 0/98 [00:00<?, ?it/s]iter called
iter called                                                   
epoch      trn_loss   val_loss   accuracy   
    0      nan        nan        0.1       
  0%|          | 0/98 [00:00<?, ?it/s]iter called
iter called                                              
    1      nan        nan        0.1       

CPU times: user 1min 51s, sys: 2min 59s, total: 4min 51s
Wall time: 47.8 s


[array([nan]), 0.1]

### Fastai DataLoader - Torch Transforms

In [34]:
tfms = torch_tfms(sz, to_numpy=True)

In [35]:
fdl_tds_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)

In [36]:
fdl_tds_data.trn_ds.open_fn = Image.open
fdl_tds_data.val_ds.open_fn = Image.open

In [37]:
learn = get_learner(fdl_tds_data)

In [38]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                     
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 2min 55s, sys: 3min 29s, total: 6min 25s
Wall time: 2min 12s


[array([nan]), 0.1]

### Torch DataLoader - Fastai Datasets

In [39]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=workers)
tdl_fds = fdl.trn_ds, fdl.val_ds
torch_data = torch_loader(str(PATH), tdl_fds)

In [40]:
learn = get_learner(torch_data)

In [41]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                     
    0      nan        nan        0.1       
    1      nan        nan        0.1                     

CPU times: user 17.1 s, sys: 8.8 s, total: 25.9 s
Wall time: 26.4 s


[array([nan]), 0.1]

### Fastai DataLoader - Torch Dataset - Same as Torch Transforms

In [ ]:
# from fastai.dataset import FilesArrayDataset, folder_source, read_dir

In [45]:
tfms = torch_tfms(sz, to_numpy=True)
dsets = torch_ds(PATH, tfms)

In [46]:
fdl_tds_data = ImageData(PATH, list(dsets)+[None,None,None,None], bs, num_workers=workers, classes=dsets[0].classes)

In [47]:
learn = get_learner(fdl_tds_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)